### Construction of the initial set

In [ ]:
cd GNAT

NCBI Gene: query = "(GNAT) AND (Pseudomonas phage)" -> 34 hits

See also `GNAT_Gene.txt`

In [ ]:
cat GNAT_Gene.txt

Restricting to ICTV viruses -> 32 hits

Manual cut-and-paste into GNAT_Nucl.fasta

In [ ]:
head GNAT_Nucl.fasta

In [ ]:
grep -c '>' GNAT_Nucl.fasta

### Constructing HMM

In [ ]:
clustalo -i GNAT_Nucl.fasta -o GNAT.sto --outfmt=st -v --force

In [ ]:
hmmbuild GNAT.hmm GNAT.sto

### nhmmer

In [ ]:
cd ..

In [ ]:
genome_db="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/AllGenBank.gb"

In [ ]:
nhmmer -E 0.001 -o ./GNAT/ICTV_nhmmer_GNAT.out \
    --tblout ./GNAT/ICTV_nhmmer_GNAT.out.table \
    ./GNAT/GNAT.hmm $genome_db
cat ./GNAT/ICTV_nhmmer_GNAT.out

In [ ]:
grep -c ! ./GNAT/ICTV_nhmmer_GNAT.out
grep -c ? ./GNAT/ICTV_nhmmer_GNAT.out

57 significant hits, including 2 LUZ19 proteins

Quite some phages of the phikmv genus appear twice (e.g. PT2, PT5, LUZ19, phiKF77, PAXYB1, RLP, MPK7), once for a sequence at front of the genome (~ 7000 bp) and once at the back (~ 21000 bp)

Also some phages of the uliginvirus genus appear twice (Achelous, Nerthus) or are close to significant (Alpheus)

### Count statistics and species assignment

In [ ]:
cd GNAT

In [ ]:
metadata="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/ICTV_Metadata_raw.txt"

In [ ]:
cat ICTV_nhmmer_GNAT.out.table

In [ ]:
accNr=$(grep -vE '^#' ICTV_nhmmer_GNAT.out.table | cut -c -9)
for nr in $accNr
do
grep $nr $metadata | cut -f 10 >> species
grep $nr $metadata | cut -f 8 >> genus
echo $nr >> accession
done
paste species accession genus > nhmmer.out.species
rm species
rm genus
rm accession
cat nhmmer.out.species

In [ ]:
sort nhmmer.out.species | uniq > counted.species
sort nhmmer.out.species | uniq -c | tr -s ' ' '\t' | cut -f 2 > counts.raw
paste counted.species counts.raw > counts
rm counts.raw
rm counted.species
cat counts

In [ ]:
cut -f 3 counts | sort | uniq -c

In [ ]:
wc -l counts

In [ ]:
cut -f 2 GNAT_Gene.txt | tail -n +2 | sort | uniq > species
while read line
do
if [ -z $(grep "$line" counts | cut -f 3) ]
then
echo "Non-ICTV" >> genus
else
grep "$line" counts | cut -f 3 >> genus
fi
done<species
cut -f 2 GNAT_Gene.txt | tail -n +2 | sort | uniq -c | tr -s ' ' '\t' | cut -f 2 > species.counts
paste species genus species.counts > nhmmer.in
rm genus
rm species.counts
rm species
cat nhmmer.in

In [ ]:
comm -23 <(cut -f 1 counts) <(cut -f 1 nhmmer.in) > nhmmer.new.species
while read s
do
grep "$s" counts | cut -f 3 >> nhmmer.new.genus
done<nhmmer.new.species
paste nhmmer.new.species nhmmer.new.genus > nhmmer.new
rm nhmmer.new.genus
rm nhmmer.new.species
cat nhmmer.new

In [ ]:
cut -f 2 nhmmer.new | sort | uniq -c